# Gaming Cross Validation and Hyperparameter Search

## Introduction

Cross validation is one of the most fundamental concepts in machine learning because it's a large part of what makes the whole process of doing ML work (IMO that and sampling). To understand why we need cross validation, and what it is, let's take a look at a realistic business use case for regression bourne out by the Linux Gaming Survey dataset.

In [ ]:
import pandas as pd
survey = pd.read_csv(
    "../input/BoilingSteam_com_LinuxGamersSurvey_Q1_2016_Public_Sharing_Only.csv"
).loc[1:]

import numpy as np

spend = survey.loc[:, ['LinuxGamingHoursPerWeek', 'LinuxGamingSpendingPerMonth']].dropna()
spend = spend.assign(
    LinuxGamingHoursPerWeek=spend.LinuxGamingHoursPerWeek.map(lambda v: int(v) if str.isdigit(v) else np.nan),
    LinuxGamingSpendingPerMonth=spend.LinuxGamingSpendingPerMonth.map(lambda v: float(v) if str.isdecimal(v) else np.nan)
).dropna()

## Problem Formulation

Suppose that we are a company named e.g. "Not-Valve Software" that interested in understanding Linux gamers. In particular, we wonder whether or not we can extrapolate the amount of money a Linux gamer will spend per month based on the amount of hours that said Linux gamer puts into gaming per week.

Looking at the pure distribution from this survey, the correlation between these two things is actually (surprisingly, to me) very low:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

sns.jointplot(spend['LinuxGamingHoursPerWeek'], spend['LinuxGamingSpendingPerMonth'])

Let's try to use linear regression to get a better handle on how this data is shaped. On average, does spending increase with gaming, on average?

In [ ]:
hours = spend['LinuxGamingHoursPerWeek'].values[:, np.newaxis]
y = spend['LinuxGamingSpendingPerMonth'].values[:, np.newaxis]

In [ ]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

def predict(degree):
    poly = PolynomialFeatures(degree=degree)
    hours_poly = poly.fit_transform(hours)
    clf = LinearRegression()
    clf.fit(hours_poly, y)
    return clf

In [ ]:
y_hat = predict(1).predict(PolynomialFeatures(degree=1).fit_transform(hours))
sortorder = np.argsort(spend['LinuxGamingHoursPerWeek'].values)
s = spend['LinuxGamingHoursPerWeek'].values[sortorder]
y_hat = y_hat[sortorder]

import matplotlib.pyplot as plt
plt.plot(s, y_hat)
plt.scatter(spend['LinuxGamingHoursPerWeek'], spend['LinuxGamingSpendingPerMonth'], color='black')

The answer seems to be yes, but for the vast majority of people the difference is relatively minor.

Perhaps the relationship between gaming and buying is non-linear, with much bigger gamers tending to spend not much more money than lower-volume gamers?

We can use a technique like polynomial regression to try and tease out more causality from this dataset in this direction. The problem is that the more degrees we add to our function, the better it will seem to perform on the whole of the data, as a rule. After all, it's hard to imagine making a model more complex decreasing your predictive strength on the data being trained on!

But we also clearly have a sense of when there is clearly too much going on. For example, here's what polynomial regression of degree 10 looks like:

In [ ]:
y_hat = predict(10).predict(PolynomialFeatures(degree=10).fit_transform(hours))
sortorder = np.argsort(spend['LinuxGamingHoursPerWeek'].values)
s = spend['LinuxGamingHoursPerWeek'].values[sortorder]
y_hat = y_hat[sortorder]

import matplotlib.pyplot as plt
plt.plot(s, y_hat)
plt.scatter(spend['LinuxGamingHoursPerWeek'], spend['LinuxGamingSpendingPerMonth'], color='black')

This is clearly a bogus predictor. The spike at around 60 hours per week is obviously an instance of overfitting: that is, of our model wanting to minimize phantom distances that happen to occur in the data but which have no relationship to reality.

The problem is that on the face of it, this is a "better" model. It's better because for this data it performs better. For example, if we take the Mean Squared Errors, a measure of model fitness, of either model, we fwill ind that the 10-degree one has a smaller mean error than the 1-degree one.

In this case it's pretty easy for us to solve this issue by generating models of varying numeric compexity and examining them to determine which one has the best fit-simplicity tradeoff. But that's only easy to do because we are using just one variable. If the dataset had many variables to use, then this becomes much harder to do.

In general, running a model many, many times by hand is tedious. To solve this fit issue, we will use a fundamental tool: cross validation.

## Cross validation

Cross validation is simply splitting a dataset into two components: a train dataset, which the model is trained on, and a test dataset, which the model is tested on. For example, a default split between these two parts of the dataset is 80-20. A model will be trained on a random 80% of the dataset; then we will evaluate how well it did using the remaining 20%.

Since that 20% was never seen by the model during training, it is not optimized for, and we can except model performance "in the wild" to be closely approximated by model performance on our training data!

This is cross validation in a nutshell. It's a really simple but critically important idea.

Let's apply this idea to our regression problem.

In [ ]:
n = len(y)
ratio = 0.8
pivot = int(np.round(0.8 * n))
train = spend.iloc[:pivot]
test = spend.iloc[pivot:]

train_X = train['LinuxGamingHoursPerWeek'].values[:, np.newaxis]
train_y = train['LinuxGamingSpendingPerMonth'].values[:, np.newaxis]
test_X = test['LinuxGamingHoursPerWeek'].values[:, np.newaxis]
test_y = test['LinuxGamingSpendingPerMonth'].values[:, np.newaxis]

In [ ]:
test_y_hat= (LinearRegression()
     .fit(PolynomialFeatures(degree=10).fit_transform(train_X), train_y)
     .predict(PolynomialFeatures(degree=10).fit_transform(test_X))
)

The mean squared error of the degree-10 model is >1300.

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(test_y, test_y_hat)

When we look at what model predicts for the points in our dataset, it becomes obvious why performance is so dismal. The model has introduced a weird spike way up to 150 to account for a phantom increase which is present in an outlier in the training dataset but which is absent in the test. Again, it's very immediate from inspecting this plot that things are going wrong.

In [ ]:
sortorder = np.argsort(test_X.flatten())
test_X_p = test_X.flatten()[sortorder]
test_y_hat_p = test_y_hat[sortorder]
plt.scatter(test_X, test_y, color='black')
plt.plot(test_X_p, test_y_hat_p)

On the other hand, when we take the mean squared error of a complexity-2 polynomial regression algorithm, we get **less than half the mean squared error** on the test dataset.

In [ ]:
test_y_hat= (LinearRegression()
     .fit(PolynomialFeatures(degree=2).fit_transform(train_X), train_y)
     .predict(PolynomialFeatures(degree=2).fit_transform(test_X))
)

mean_squared_error(test_y, test_y_hat)

Indeed, the plot shows a vastly improved and saner fit curve:

In [ ]:
sortorder = np.argsort(test_X.flatten())
test_X_p = test_X.flatten()[sortorder]
test_y_hat_p = test_y_hat[sortorder]
plt.scatter(test_X, test_y, color='black')
plt.plot(test_X_p, test_y_hat_p)

Clearly this is a better and far more usable model!

## Hyperparameter search

Which cross validation now in our toolbelt, we can approach the problem again: what polynomial model performs best when applied to our problem?

We expect cross validation to be a good estimator on model performance against never-before-seen data. Hence we can use cross validation&mdash;that is, checking the mean squared error of the classifier applied to the test dataset&mdash;for, say, every degree polynomial regression function between 1 and 10. Clearly our best-performing model will be somewhere in there!

This is known as a hyperparameter search. Hyperparameter searches are important because they are, effectively, how we go about finding the most useful and effective model in a series of possible models controlled by some magic number (the so-named "hyperparameter").

In this case we'll do the search by hand (`scikit-learn` has extensive hyperparameter search facilities that vary model-to-model). For each polynomial regression model between degree 1 and degree 10, we will compute the mean squared error of that model as applied to our cross validation test dataset. This will finally tell us once and for all whether a linear model or a simple polynomial model is best!

In [ ]:
mses = [mean_squared_error(
    test_y, 
    (LinearRegression()
        .fit(PolynomialFeatures(degree=n).fit_transform(train_X), train_y)
        .predict(PolynomialFeatures(degree=n).fit_transform(test_X)))
 ) for n in range(1, 11)]

In [ ]:
pd.Series(mses, index=range(1, 11)).plot.line(title='CV MSE per N(Coefficients)')

## Conclusion

As this chart shows, cross-validated Mean Squared Error climbs *upwards* with every step up we take in model complexity. This chart makes it abundantly clear that the best model performance is achieved when the model is the simplest one possible: a linear one. Anything else we do will worsen our real-world results!

That concludes our quick introduction to cross validation and hyperparameter search. Hopefully you will find this kernel helpful in your own learning!